In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import math

def initialize_s(qc, qubits):
    """Apply H-gate to all qubits to create superposition |s>"""
    for q in qubits:
        qc.h(q)

def oracle(qc, qubits, target_state):
    """
    Marks the target state by flipping its phase.
    Logic:
    1. Flip '0' bits in target to '1' using X gates.
    2. Apply Multi-Controlled Z (MCZ) to mark the |11...1> state.
    3. Flip the bits back.
    """
    n = len(qubits)
    # Reverse string because Qiskit reads right-to-left (q0 is far right)
    target_rev = target_state[::-1]
    
    # 1. Flip qubits to match |11...1>
    for i in range(n):
        if target_rev[i] == '0':
            qc.x(i)
            
    # 2. Apply MCZ (Multi-Controlled Z)
    # Qiskit's mcp(pi) acts as a controlled-Z
    qc.h(n-1) # Turn last qubit to X-basis
    qc.mcx(qubits[:-1], n-1) # Multi-controlled X (Toffoli)
    qc.h(n-1) # Turn back to Z-basis
    
    # 3. Un-flip qubits
    for i in range(n):
        if target_rev[i] == '0':
            qc.x(i)

def diffuser(qc, qubits):
    """
    Amplifies the marked amplitude.
    Operator: 2|s><s| - I
    """
    n = len(qubits)
    
    # 1. Apply H to all
    qc.h(qubits)
    
    # 2. Apply X to all
    qc.x(qubits)
    
    # 3. Apply MCZ (Multi-Controlled Z)
    qc.h(n-1)
    qc.mcx(qubits[:-1], n-1)
    qc.h(n-1)
    
    # 4. Apply X to all
    qc.x(qubits)
    
    # 5. Apply H to all
    qc.h(qubits)

def run_grover(n, target_state):
    # Calculate optimal iterations: pi/4 * sqrt(N)
    N = 2**n
    iterations = int(math.floor((np.pi / 4) * math.sqrt(N)))
    print(f"Running Grover's for {n} qubits.")
    print(f"Search Space size: {N}")
    print(f"Target: |{target_state}>")
    print(f"Optimal Iterations: {iterations}")

    # Create Circuit
    qc = QuantumCircuit(n)
    qubits = list(range(n))

    # 1. Initialize
    initialize_s(qc, qubits)

    # Loop Step
    for _ in range(iterations):
        qc.barrier() # Visual separator
        oracle(qc, qubits, target_state)
        qc.barrier()
        diffuser(qc, qubits)

    # Measurement
    qc.measure_all()

    # Simulation
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1024).result()
    counts = result.get_counts()

    # Display Result
    print(f"\nResults: {counts}")
    most_frequent = max(counts, key=counts.get)
    print(f"Most frequent measurement: |{most_frequent}>")
    
    # Draw the circuit
    print("\nCircuit Diagram:")
    print(qc.draw(output='text'))

# --- Configuration ---
n_qubits = 3
target_string = '101' # Change this to search for different items

run_grover(n_qubits, target_string)

Running Grover's for 3 qubits.
Search Space size: 8
Target: |101>
Optimal Iterations: 2

Results: {'101': 970, '100': 6, '111': 9, '000': 10, '110': 12, '010': 8, '001': 5, '011': 4}
Most frequent measurement: |101>

Circuit Diagram:
        ┌───┐ ░                 ░ ┌───┐┌───┐          ┌───┐┌───┐      ░      »
   q_0: ┤ H ├─░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░──────»
        ├───┤ ░ ┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  ├───┤├───┤      ░ ┌───┐»
   q_1: ┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─┤ X ├»
        ├───┤ ░ ├───┤┌─┴─┐├───┤ ░ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ░ ├───┤»
   q_2: ┤ H ├─░─┤ H ├┤ X ├┤ H ├─░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─░─┤ H ├»
        └───┘ ░ └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘└───┘└───┘└───┘ ░ └───┘»
meas: 3/═════════════════════════════════════════════════════════════════════»
                                                                             »
«                   ░ ┌───┐┌───┐          ┌───┐┌───┐   